In [13]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [12]:
# # Initialize PyMongo to work with MongoDBs
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)

In [13]:
# # Define database and collection
# db = client.mars_db
# collection = db.news

In [3]:
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/85.0.4183.87/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/kirstiekjones/.wdm/drivers/chromedriver/mac64/85.0.4183.87]


In [4]:
# URL of NASA page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [5]:
# # Retrieve page with the requests module
# response = requests.get(url)

# # Create BeautifulSoup object; parse with 'lxml'
# soup = BeautifulSoup(response.text, 'lxml')

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [15]:
# Retrieve the parent divs for all articles
results = soup.find_all('li', class_="slide")

# loop over results to get article data
for result in results:
    try:
        # scrape the article header 
        news_title = result.find('div', class_='content_title').text

        # scrape the article subheader
        news_p = result.find('div', class_='article_teaser_body').text
        
        # print article data
        print('-----------------')
        print(news_title)
        print(news_p)

#         # Dictionary to be inserted into MongoDB
#         post = {
#             'news_title': news_title,
#             'news_p': news_p,
#         }

#         # Insert dictionary into MongoDB as a document
#         collection.insert_one(post)
    except:
        pass

-----------------
NASA Readies Perseverance Mars Rover's Earthly Twin 
Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.
-----------------
NASA Engineers Checking InSight's Weather Sensors
An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.
-----------------
Follow NASA's Perseverance Rover in Real Time on Its Way to Mars
A crisply rendered web application can show you where the agency's Mars 2020 mission is right now as it makes its way to the Red Planet for a Feb. 18, 2021, landing.
-----------------
NASA Establishes Board to Initially Review Mars Sample Return Plans
The board will assist with analysis of current plans and goals for one of the most difficult missions humanity has ever undertaken.
-----------------
NASA's Ingenuity Mars Helicopter Recharges Its Batteries in Flight
Heade

In [6]:
# URL of JPL page to be scraped
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(featured_image_url)

In [24]:
#browser.quit()

In [7]:
# Navigate to id for button "Full Image" & click
browser.find_by_id('full_image').click()

In [8]:
#Click button "More Info"
browser.links.find_by_partial_text('more info').click()

In [9]:
# Retrieve Featured image from page
soup = BeautifulSoup(browser.html, "html.parser")


In [11]:
featured_image_url = f"https://www.jpl.nasa.gov{soup.find('img', class_='main_image')['src']}"
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18811_hires.jpg'

In [22]:
mars_facts = pd.read_html('https://space-facts.com/mars/')[0]

#set index to 0
#to_html
mars_facts.columns=["Description", "Data"]

mars_facts.set_index("Description", inplace=False)

Data
Description                                        
Equatorial Diameter:                       6,792 km
Polar Diameter:                            6,752 km
Mass:                 6.39 × 10^23 kg (0.11 Earths)
Moons:                          2 (Phobos & Deimos)
Orbit Distance:            227,943,824 km (1.38 AU)
Orbit Period:                  687 days (1.9 years)
Surface Temperature:                   -87 to -5 °C
First Record:                     2nd millennium BC
Recorded By:                   Egyptian astronomers

In [24]:
mars_facts.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Data</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium B

In [20]:
# URL of Mars Facts page to be scraped
mars_facts_url = 'https://space-facts.com/mars/'
browser.visit(mars_facts_url)

In [21]:
# Define database and collection
db = client.mars_db
collection = db.mars_facts

In [22]:
# Retrieve table facts from page
soup = BeautifulSoup(browser.html, "html.parser")
mars_facts = soup.find("table", class_="tablepress tablepress-id-p-mars")

for fact in mars_facts:
    try:
        # scrape the equatorial diameter
        equa_diameter = fact.find("tr", class_="row-1 odd").text
        
        # scrape the polar diameter
        pol_diameter = fact.find("tr", class_="row-2 even").text
        
        # scrape the mass
        mars_mass = fact.find("tr", class_="row-3 odd").text
        
        # scrape the moons
        mars_moons = fact.find("tr", class_="row-4 even").text
        
        # scrape the orbit distance
        orbit_dist = fact.find("tr", class_="row-5 odd").text
        
        # scrape the orbit period
        orbit_per = fact.find("tr", class_="row-6 even").text
        
        # scrape the surface temperature
        surface_temp = fact.find("tr", class_="row-7 odd").text
        
        # scrape the first record
        first_rec = fact.find("tr", class_="row-8 even").text
        
        # scrape recorded by
        recorded_by = fact.find("tr", class_="row-9 odd").text
        
        # print table data
        print('-----------------')
        print(equa_diameter)
        print(pol_diameter)
        print(mars_mass)
        print(mars_moons)
        print(orbit_dist)
        print(orbit_per)
        print(surface_temp)
        print(first_rec)
        print(recorded_by)
        
        
        
#         # Dictionary to be inserted into MongoDB
#         post = {
#             'equa_diameter': equatorial_diameter,
#             'pol_diameter': polar_diameter,
#             'mars_mass': mass,
#             'mars_moons': moons,
#             'orbit_dist': orbit_distance,
#             'orbit_per': orbit_period,
#             'surface_temp': surface_temp,
#             'first_rec': first_record,
#             'recorded_by': recorded_by
#         }
        
#         # insert dictionary into MongoDB as a document
#         collection.insert_one(post)
    except:
        pass

-----------------
Equatorial Diameter:6,792 km
Polar Diameter:6,752 km
Mass:6.39 × 10^23 kg (0.11 Earths)
Moons:2 (Phobos & Deimos)
Orbit Distance:227,943,824 km (1.38 AU)
Orbit Period:687 days (1.9 years)
Surface Temperature: -87 to -5 °C
First Record:2nd millennium BC
Recorded By:Egyptian astronomers


In [5]:
# URL of Mars hemispheres page to be scraped
mars_hemispheres = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(mars_hemispheres)


In [7]:
# Navigate to Cerberus Hemisphere link & click
hemispheres = browser.find_by_css('h3')
hems_list =[]


for hemisphere in range(len(hemispheres)):
    mars_hems_dict = {}
    #print(title)
    title = browser.find_by_css('a.product-item h3')[hemisphere].text
     # create empty dict to store title & URL , then append dicts to larger list
    mars_hems_dict["title"]= title
    browser.find_by_css('a.product-item h3')[hemisphere].click()
    
    # Navigate to image download 
    mars_image = browser.find_by_text('Sample').first['href']
    # store img in dict
    mars_hems_dict["image"]=mars_image
    # store dictionary items into hems_list
    hems_list.append(mars_hems_dict)
    
    browser.back()
    

In [8]:
print(hems_list)

[{'title': 'Cerberus Hemisphere Enhanced', 'image': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'image': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'image': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'image': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [30]:
# # Navigate to image download 
# browser.find_by_text('Sample').first['href']

'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'